In [1]:
# Define the folder path where the data files are located
data_folder = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections"

# List of all the file names to be merged
file_names = ['3. Employment&Income_Household-HouseholdMember_Level.csv']


In [28]:
import pandas as pd
import os

# Define the folder path where the data files are located
data_folder = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections"

# List of all the file names to be merged
file_names = ['17. ClimateChange_Household-HouseholdMember_Level.csv']

In [30]:
# for a single unique identifier

import os
import pandas as pd

# Initialize an empty DataFrame to hold the merged data
merged_df = pd.DataFrame()

# Iterate over each file in the list, read it, and concatenate it to the merged DataFrame
for file_name in file_names:
    file_path = os.path.join(data_folder, file_name)  # Create the full file path
    print(f"Looking for file: {file_path}")  # Debug print
    if os.path.exists(file_path):
        print(f"File found: {file_path}")  # Debug print
        df = pd.read_csv(file_path, dtype=str)  # Read the CSV file with all columns as strings to handle mixed data types

        # Convert numeric columns to double (if possible)
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors='raise', downcast='float')
            except ValueError:
                # Keep non-convertible columns as strings
                pass

        merged_df = pd.concat([merged_df, df], ignore_index=True)  # Concatenate the current file's data to the merged DataFrame
    else:
        print(f"File not found: {file_path}")  # Debug print

# Proceed only if merged_df is not empty
if not merged_df.empty:
    # Define the merge keys
    merge_keys = ['HID']
    # Filter out merge keys that are not in the DataFrame
    available_merge_keys = [key for key in merge_keys if key in merged_df.columns]

    # Function to handle aggregation of non-merge key columns
    def aggregate_data(group):
        # Initialize an empty dictionary to hold aggregated results
        agg_dict = {}
        for col in group.columns:
            if col in available_merge_keys:
                # Keep merge keys as they are
                agg_dict[col] = group[col].iloc[0]
            else:
                # For numeric columns, sum the values
                if pd.api.types.is_numeric_dtype(group[col]):
                    agg_dict[col] = group[col].sum()
                else:
                    # For string columns, concatenate all values
                    agg_dict[col] = ', '.join(group[col].dropna().astype(str).unique())
        return pd.Series(agg_dict)

    # Check for initial duplicates
    initial_duplicates = merged_df[merged_df.duplicated(subset=available_merge_keys, keep=False)]
    if not initial_duplicates.empty:
        print(f"Initial duplicates found based on keys {available_merge_keys}:")
        print(initial_duplicates)

    # Handle rows with missing keys separately
    complete_keys_df = merged_df.dropna(subset=available_merge_keys)
    incomplete_keys_df = merged_df[merged_df[available_merge_keys].isnull().any(axis=1)]

    # Drop rows with missing merge keys
    merged_df.dropna(subset=available_merge_keys, inplace=True)

    # Apply aggregation function to each group for rows with complete keys
    if not merged_df.empty:
        merged_df = merged_df.groupby(available_merge_keys).apply(aggregate_data).reset_index(drop=True)

    # Check for duplicates after aggregation
    post_aggregation_duplicates = merged_df[merged_df.duplicated(subset=available_merge_keys, keep=False)]
    if not post_aggregation_duplicates.empty:
        print(f"Post-aggregation duplicates found based on keys {available_merge_keys}:")
        print(post_aggregation_duplicates)

    # Reorder columns to have available merge keys first and then the rest alphabetically
    remaining_columns = [col for col in merged_df.columns if col not in available_merge_keys]  # Get columns excluding available merge keys
    # Sort the remaining columns alphabetically; put columns starting with digits at the end
    sorted_columns = sorted(remaining_columns, key=lambda x: (x[0].isdigit(), x))

    # Define the final column order with available merge keys first
    column_order = available_merge_keys + sorted_columns
    # Reorder the DataFrame columns according to the defined order
    merged_df = merged_df[column_order]

    # Remove columns that are entirely empty
    merged_df.dropna(axis=1, how='all', inplace=True)
    # Remove rows that are entirely empty
    merged_df.dropna(axis=0, how='all', inplace=True)

    # Append rows with incomplete keys to the end of the DataFrame
    final_df = pd.concat([merged_df, incomplete_keys_df], ignore_index=True)

    # Save the merged and processed data to a new CSV file
    final_df.to_csv('17. ClimateChange_Household-HouseholdMember_Level Updated.csv', index=False)
    print("Merged data saved successfully!")
else:
    print("No data to merge and save.")


Looking for file: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\17. ClimateChange_Household-HouseholdMember_Level.csv
File found: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\17. ClimateChange_Household-HouseholdMember_Level.csv
Initial duplicates found based on keys ['HID']:
               HID  C_HH_NUM  D_ID  M_ID  PCC_Atti  PCC_CiAR  PCC_CiT  \
0     1.010101e+10       4.0   1.0   1.0       3.0       2.0      1.0   
1     1.010101e+10       8.0   1.0   1.0       1.0       2.0      1.0   
2     1.010101e+10      15.0   1.0   1.0       0.0       2.0      1.0   
3     1.010101e+10      19.0   1.0   1.0       4.0       2.0      1.0   
4     1.010101e+10      23.0   1.0   1.0       1.0       2.0      1.0   
...            ...       ...   ...   ...       ...       ...      ...   
1879  3.194774e+10      22.0  19.0  75.0       NaN       NaN      NaN   
1880  3.194774e+10      24.0  19.0  75.0       NaN       NaN      NaN   
1881 

In [25]:
# Initialize an empty DataFrame to hold the merged data
merged_df = pd.DataFrame()

# Iterate over each file in the list, read it, and concatenate it to the merged DataFrame
for file_name in file_names:
    file_path = os.path.join(data_folder, file_name)  # Create the full file path
    print(f"Looking for file: {file_path}")  # Debug print
    if os.path.exists(file_path):
        print(f"File found: {file_path}")  # Debug print
        df = pd.read_csv(file_path, dtype=str)  # Read the CSV file with all columns as strings to handle mixed data types

        # Convert numeric columns to double (if possible)
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors='raise', downcast='float')
            except ValueError:
                # Keep non-convertible columns as strings
                pass

        merged_df = pd.concat([merged_df, df], ignore_index=True)  # Concatenate the current file's data to the merged DataFrame
    else:
        print(f"File not found: {file_path}")  # Debug print

# Proceed only if merged_df is not empty
if not merged_df.empty:
    # Define the merge keys
    merge_keys = ['HID', 'PID']
    # Filter out merge keys that are not in the DataFrame
    available_merge_keys = [key for key in merge_keys if key in merged_df.columns]

    # Function to handle aggregation of non-merge key columns
    def aggregate_data(group):
        # Initialize an empty dictionary to hold aggregated results
        agg_dict = {}
        for col in group.columns:
            if col in available_merge_keys:
                # Keep merge keys as they are
                agg_dict[col] = group[col].iloc[0]
            else:
                # For numeric columns, sum the values
                if pd.api.types.is_numeric_dtype(group[col]):
                    agg_dict[col] = group[col].sum()
                else:
                    # For string columns, concatenate all values
                    agg_dict[col] = ', '.join(group[col].dropna().astype(str).unique())
        return pd.Series(agg_dict)

    # Handle rows with missing keys separately
    complete_keys_df = merged_df.dropna(subset=available_merge_keys)
    incomplete_keys_df = merged_df[merged_df[available_merge_keys].isnull().any(axis=1)]

    # Drop rows with missing merge keys
    merged_df.dropna(subset=available_merge_keys, inplace=True)

    # Apply aggregation function to each group for rows with complete keys
    if not merged_df.empty:
        merged_df = merged_df.groupby(available_merge_keys).apply(aggregate_data).reset_index(drop=True)

    # Reorder columns to have available merge keys first and then the rest alphabetically
    remaining_columns = [col for col in merged_df.columns if col not in available_merge_keys]  # Get columns excluding available merge keys
    # Sort the remaining columns alphabetically; put columns starting with digits at the end
    sorted_columns = sorted(remaining_columns, key=lambda x: (x[0].isdigit(), x))

    # Define the final column order with available merge keys first
    column_order = available_merge_keys + sorted_columns
    # Reorder the DataFrame columns according to the defined order
    complete_keys_df = complete_keys_df[column_order]

    # Remove columns that are entirely empty
    complete_keys_df.dropna(axis=1, how='all', inplace=True)
    # Remove rows that are entirely empty
    complete_keys_df.dropna(axis=0, how='all', inplace=True)

    # Append rows with incomplete keys to the end of the DataFrame
    final_df = pd.concat([complete_keys_df, incomplete_keys_df], ignore_index=True)

    # Save the merged and processed data to a new CSV file
    final_df.to_csv('15. ExtendedFamily_Household-HouseholdMember_Level Updated.csv', index=False)
    print("Merged data saved successfully!")
else:
    print("No data to merge and save.")

Looking for file: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\16. AccessToExtension_Household-HouseholdMember_Level.csv
File found: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\16. AccessToExtension_Household-HouseholdMember_Level.csv


ValueError: No group keys passed!

In [ ]:
import pandas as pd

# Define the file path and the keys to check
file_path = r"C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\3. Agriculture_Household-HouseholdMember_Level Updated.csv"
merge_keys = ['HID', 'PID']

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path, low_memory=False)

# Check for duplicates based on the merge keys
duplicates = df[df.duplicated(subset=merge_keys, keep=False)]

# Print the results
if not duplicates.empty:
    print(f"Found duplicates based on the keys {merge_keys}:")
    print(duplicates)
    print(f"\nTotal number of duplicate rows: {duplicates.shape[0]}")
else:
    print(f"No duplicates found based on the keys {merge_keys}.")
    

In [15]:
# HID, PID, PlotID, CropID, Season Level

# Initialize an empty DataFrame to hold the merged data
merged_df = pd.DataFrame()

# Iterate over each file in the list, read it, and concatenate it to the merged DataFrame
for file_name in file_names:
    file_path = os.path.join(data_folder, file_name)  # Create the full file path
    print(f"Looking for file: {file_path}")  # Debug print
    if os.path.exists(file_path):
        print(f"File found: {file_path}")  # Debug print
        df = pd.read_csv(file_path, dtype=str)  # Read the CSV file with all columns as strings to handle mixed data types

        # Convert numeric columns to double (if possible)
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors='raise', downcast='float')
            except ValueError:
                # Keep non-convertible columns as strings
                pass

        merged_df = pd.concat([merged_df, df], ignore_index=True)  # Concatenate the current file's data to the merged DataFrame
    else:
        print(f"File not found: {file_path}")  # Debug print

# Proceed only if merged_df is not empty
if not merged_df.empty:
    # Define the merge keys
    merge_keys = ['HID', 'PID', 'Plt_ID', 'Crop_ID', 'Season']
    # Filter out merge keys that are not in the DataFrame
    available_merge_keys = [key for key in merge_keys if key in merged_df.columns]

    # Function to handle aggregation of non-merge key columns
    def aggregate_data(group):
        # Initialize an empty dictionary to hold aggregated results
        agg_dict = {}
        for col in group.columns:
            if col in available_merge_keys:
                # Keep merge keys as they are
                agg_dict[col] = group[col].iloc[0]
            else:
                # For numeric columns, sum the values
                if pd.api.types.is_numeric_dtype(group[col]):
                    agg_dict[col] = group[col].sum()
                else:
                    # For string columns, concatenate all values
                    agg_dict[col] = ', '.join(group[col].dropna().astype(str).unique())
        return pd.Series(agg_dict)

    # Handle rows with missing keys separately
    complete_keys_df = merged_df.dropna(subset=available_merge_keys)
    incomplete_keys_df = merged_df[merged_df[available_merge_keys].isnull().any(axis=1)]

    # Drop rows with missing merge keys
    merged_df.dropna(subset=available_merge_keys, inplace=True)

    # Apply aggregation function to each group for rows with complete keys
    if not merged_df.empty:
        merged_df = merged_df.groupby(available_merge_keys).apply(aggregate_data).reset_index(drop=True)

    # Reorder columns to have available merge keys first and then the rest alphabetically
    remaining_columns = [col for col in merged_df.columns if col not in available_merge_keys]  # Get columns excluding available merge keys
    # Sort the remaining columns alphabetically; put columns starting with digits at the end
    sorted_columns = sorted(remaining_columns, key=lambda x: (x[0].isdigit(), x))

    # Define the final column order with available merge keys first
    column_order = available_merge_keys + sorted_columns
    # Reorder the DataFrame columns according to the defined order
    complete_keys_df = complete_keys_df[column_order]

    # Remove columns that are entirely empty
    complete_keys_df.dropna(axis=1, how='all', inplace=True)
    # Remove rows that are entirely empty
    complete_keys_df.dropna(axis=0, how='all', inplace=True)

    # Append rows with incomplete keys to the end of the DataFrame
    final_df = pd.concat([complete_keys_df, incomplete_keys_df], ignore_index=True)

    # Save the merged and processed data to a new CSV file
    final_df.to_csv('3.1. Agriculture_Household-HouseholdMember-Plot-Crop-Season_Level Updated.csv', index=False)
    print("Merged data saved successfully!")
else:
    print("No data to merge and save.")


Looking for file: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\3.1. Agriculture_Household-HouseholdMember-Plot-Crop-Season_Level.csv
File found: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\3.1. Agriculture_Household-HouseholdMember-Plot-Crop-Season_Level.csv
Merged data saved successfully!


In [5]:
# HID, Season, ShockID Level

# Initialize an empty DataFrame to hold the merged data
merged_df = pd.DataFrame()

# Iterate over each file in the list, read it, and concatenate it to the merged DataFrame
for file_name in file_names:
    file_path = os.path.join(data_folder, file_name)  # Create the full file path
    print(f"Looking for file: {file_path}")  # Debug print
    if os.path.exists(file_path):
        print(f"File found: {file_path}")  # Debug print
        df = pd.read_csv(file_path, dtype=str)  # Read the CSV file with all columns as strings to handle mixed data types

        # Convert numeric columns to double (if possible)
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col], errors='raise', downcast='float')
            except ValueError:
                # Keep non-convertible columns as strings
                pass

        merged_df = pd.concat([merged_df, df], ignore_index=True)  # Concatenate the current file's data to the merged DataFrame
    else:
        print(f"File not found: {file_path}")  # Debug print

# Proceed only if merged_df is not empty
if not merged_df.empty:
    # Define the merge keys
    merge_keys = ['HID', 'Season','Shock_ID']
    # Filter out merge keys that are not in the DataFrame
    available_merge_keys = [key for key in merge_keys if key in merged_df.columns]

    # Function to handle aggregation of non-merge key columns
    def aggregate_data(group):
        # Initialize an empty dictionary to hold aggregated results
        agg_dict = {}
        for col in group.columns:
            if col in available_merge_keys:
                # Keep merge keys as they are
                agg_dict[col] = group[col].iloc[0]
            else:
                # For numeric columns, sum the values
                if pd.api.types.is_numeric_dtype(group[col]):
                    agg_dict[col] = group[col].sum()
                else:
                    # For string columns, concatenate all values
                    agg_dict[col] = ', '.join(group[col].dropna().astype(str).unique())
        return pd.Series(agg_dict)

    # Handle rows with missing keys separately
    complete_keys_df = merged_df.dropna(subset=available_merge_keys)
    incomplete_keys_df = merged_df[merged_df[available_merge_keys].isnull().any(axis=1)]

    # Drop rows with missing merge keys
    merged_df.dropna(subset=available_merge_keys, inplace=True)

    # Apply aggregation function to each group for rows with complete keys
    if not merged_df.empty:
        merged_df = merged_df.groupby(available_merge_keys).apply(aggregate_data).reset_index(drop=True)

    # Reorder columns to have available merge keys first and then the rest alphabetically
    remaining_columns = [col for col in merged_df.columns if col not in available_merge_keys]  # Get columns excluding available merge keys
    # Sort the remaining columns alphabetically; put columns starting with digits at the end
    sorted_columns = sorted(remaining_columns, key=lambda x: (x[0].isdigit(), x))

    # Define the final column order with available merge keys first
    column_order = available_merge_keys + sorted_columns
    # Reorder the DataFrame columns according to the defined order
    complete_keys_df = complete_keys_df[column_order]

    # Remove columns that are entirely empty
    complete_keys_df.dropna(axis=1, how='all', inplace=True)
    # Remove rows that are entirely empty
    complete_keys_df.dropna(axis=0, how='all', inplace=True)

    # Append rows with incomplete keys to the end of the DataFrame
    final_df = pd.concat([complete_keys_df, incomplete_keys_df], ignore_index=True)

    # Save the merged and processed data to a new CSV file
    final_df.to_csv('8.1. EconomicEventsAndShocks_Household-Season-ShockType_Level Updated.csv', index=False)
    print("Merged data saved successfully!")
else:
    print("No data to merge and save.")




Looking for file: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\8.1. EconomicEventsAndShocks_Household-Season-ShockType_Level.csv
File found: C:\Users\warra\Desktop\Freelance\data\data\MaleMerge\3. MERGED CSV Sections\8.1. EconomicEventsAndShocks_Household-Season-ShockType_Level.csv
Merged data saved successfully!
